In [26]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pyodbc
from pandasql import sqldf
import requests
from bs4 import BeautifulSoup
import urllib.request


`Licor Temp (C)` : Temperature of the Infrared Licor 820 in degrees Celsius <br>
`xCO2 SW (wet)` : partial pressure of carbon dioxide - water, (μmol/mol) Mole fraction of CO2 in air in equilibrium with the seawater at sea surface temperature and measured humidity.  <br>
`pCO2 SW (wet)` : partial pressure of carbon dioxide - water , (uatm) Partial Pressure of CO2 in air in equilibrium with the seawater at sea surface temperature (100% humidity) <br>
`fCO2 SW (sat)` : partial pressure of carbon dioxide - water, (uatm) Fugacity of CO2 in air in equilibrium with the seawater at sea surface temperature (100% humidity) <br>
`xCO2 SW (dry)` : partial pressure of carbon dioxide - water, (μmol/mol) Mole fraction of CO2 in air in equilibrium with the seawater at sea surface temperature (dry air) <br>
`Latitude` : 47.803 <br>
`Longitude`: -122.803 <br>
https://www.ncei.noaa.gov/data/oceans/ncei/ocads/metadata/0116715.html <br>
Since there are so many NaN data on SST, we are going to find another temperature dataset, and will merge them <br>
Also, there are also many Nan data on `fCO2 SW (sat)` and `pCO2 SW (wet)`, so we are going to use `xCO2 SW (wet)` and `xCO2 SW (dry)` to check acidification. 

In [27]:
def get_urls(url):
    
    """
    Input: 
        - Takes a url as input.
    Output: 
        - Output is a list of all the scraped urls associated with the input url link.  
    """
    
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
 
    urls = []
    for link in soup.find_all('a'):
       print(link.get('href'))
   

In [28]:
url = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0116715/'
get_urls(url)


?C=N;O=D
?C=M;O=A
?C=S;O=A
?C=D;O=A
/data/oceans/ncei/ocads/data/
Dabob_122W_48N_Apr2014_Dec2014.csv
Dabob_122W_48N_Apr2014_Dec2014.xml
Dabob_122W_48N_Apr2014_Dec2014_QFLog.csv
Dabob_122W_48N_Dec2014_Jan2016.csv
Dabob_122W_48N_Dec2014_Jan2016.xml
Dabob_122W_48N_Dec2014_Jan2016_QFLog.csv
Dabob_122W_48N_Jan2016_Mar2017.csv
Dabob_122W_48N_Jan2016_Mar2017_PI_OME.pdf
Dabob_122W_48N_Jan2016_Mar2017_PI_OME.xml
Dabob_122W_48N_Jan2016_Mar2017_QFLog.csv
Dabob_122W_48N_Jul2012_Sep2012.csv
Dabob_122W_48N_Jul2012_Sep2012_PI_OME.pdf
Dabob_122W_48N_Jul2012_Sep2012_PI_OME.xml
Dabob_122W_48N_Jul2012_Sep2012_QFLog.csv
Dabob_122W_48N_Jul2019_Jun2020.csv
Dabob_122W_48N_Jul2019_Jun2020_PI_OME.pdf
Dabob_122W_48N_Jul2019_Jun2020_PI_OME.xml
Dabob_122W_48N_Jul2019_Jun2020_QFLog.csv
Dabob_122W_48N_Jul2020_Nov2021.csv
Dabob_122W_48N_Jul2020_Nov2021_PI_OME.pdf
Dabob_122W_48N_Jul2020_Nov2021_PI_OME.xml
Dabob_122W_48N_Jul2020_Nov2021_QFLog.csv
Dabob_122W_48N_Jun2011_Jul2012.csv
Dabob_122W_48N_Jun2011_Jul2012_PI_OME

In [29]:
csv_links = 'Dabob_122W_48N_Apr2014_Dec2014.csv\n\
Dabob_122W_48N_Dec2014_Jan2016.csv\n\
Dabob_122W_48N_Jan2016_Mar2017.csv\n\
Dabob_122W_48N_Jul2012_Sep2012.csv\n\
Dabob_122W_48N_Jul2019_Jun2020.csv\n\
Dabob_122W_48N_Jul2020_Nov2021.csv\n\
Dabob_122W_48N_Sep2013_Mar2014.csv\n\
Dabob_122W_48N_Jun2011_Jul2012.csv\n\
Dabob_122W_48N_May2018_Jul2019.csv'

datasets = open('csv_links.txt', 'w')
datasets.write(csv_links)
datasets.close()

In [30]:
#Csv files are saved on system for easier access:
file = open('csv_links.txt', 'r')
Lines = file.readlines()
read = []

for line in Lines:
    url = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0116715/'+str(line)
    with urllib.request.urlopen(url) as testfile, open(line.strip('\n'), 'w') as f:
        f.write(testfile.read().decode())
    


In [31]:
#2011-2012
PH1112 = pd.read_csv("Dabob_122W_48N_Jun2011_Jul2012.csv", skiprows=4)
PH1112['Date']= pd.to_datetime(PH1112['Date'])
ACID_1112 = PH1112[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_1112 = ACID_1112.replace(-999, np.NaN)

In [32]:
#2012-2012
PH1212 = pd.read_csv("Dabob_122W_48N_Jul2012_Sep2012.csv", skiprows=4)
PH1212['Date']= pd.to_datetime(PH1212['Date'])
ACID_1212 = PH1212[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_1212 = ACID_1212.replace(-999, np.NaN)

In [33]:
#2013-2014
PH1314 = pd.read_csv("Dabob_122W_48N_Sep2013_Mar2014.csv", skiprows=4)
PH1314['Date']= pd.to_datetime(PH1314['Date'])
ACID_1314 = PH1314[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_1314 = ACID_1314.replace(-999, np.NaN)

In [34]:
#2014-2014
PH1414 = pd.read_csv("Dabob_122W_48N_Apr2014_Dec2014.csv")
PH1414['Date']= pd.to_datetime(PH1414['Date'])
ACID_1414 = PH1414[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_1414 = ACID_1414.replace(-999, np.NaN)

In [35]:
#2014-2016
PH1416 = pd.read_csv("Dabob_122W_48N_Dec2014_Jan2016.csv")
PH1416['Date']= pd.to_datetime(PH1416['Date'])
ACID_1416 = PH1416[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_1416 = ACID_1416.replace(-999, np.NaN)

In [36]:
#2016-2017
PH1617 = pd.read_csv("Dabob_122W_48N_Jan2016_Mar2017.csv", skiprows=4)
PH1617['Date']= pd.to_datetime(PH1617['Date'])
ACID_1617 = PH1617[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_1617 = ACID_1617.replace(-999, np.NaN)

In [37]:
#2018-2019
PH1819 = pd.read_csv("Dabob_122W_48N_May2018_Jul2019.csv", skiprows=4)
PH1819['Date']= pd.to_datetime(PH1819['Date'])
ACID_1819 = PH1819[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_1819 = ACID_1819.replace(-999, np.NaN)

In [38]:
#2019-2020
PH1920 = pd.read_csv("Dabob_122W_48N_Jul2019_Jun2020.csv", skiprows=4)
PH1920['Date']= pd.to_datetime(PH1920['Date'])
ACID_1920 = PH1920[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_1920 = ACID_1920.replace(-999, np.NaN)

In [39]:
#2020-2021
PH2021 = pd.read_csv("Dabob_122W_48N_Jul2020_Nov2021.csv", skiprows=4)
PH2021['Date']= pd.to_datetime(PH2021['Date'])
ACID_2021 = PH2021[["Date","xCO2 SW (wet) (umol/mol)",  "xCO2 SW (dry) (umol/mol)", "Licor Temp (C)"]]
ACID_2021 = ACID_2021.replace(-999, np.NaN)

In [40]:
frames = [ACID_1112, ACID_1212, ACID_1314, ACID_1414, ACID_1416, ACID_1617, ACID_1819, ACID_1920, ACID_2021]
ACID = pd.concat(frames)

In [41]:
ACID.rename(columns={"xCO2 SW (wet) (umol/mol)":'xCO2_wet'}, inplace=True)
ACID.rename(columns={"xCO2 SW (dry) (umol/mol)":'xCO2_dry'}, inplace=True)
ACID.rename(columns={"Licor Temp (C)":'temp'}, inplace=True)
ACID

,Date,xCO2_wet,xCO2_dry,temp
0,2011-06-10,252.0,252.5,18.8
1,2011-06-10,260.2,260.6,16.4
2,2011-06-10,261.1,261.5,15.1
3,2011-06-10,275.1,275.5,14.3
4,2011-06-10,257.9,258.2,13.8
...,...,...,...,...
3676,2021-11-04,755.1,756.8,12.4
3677,2021-11-04,764.1,765.9,12.5
3678,2021-11-04,767.9,769.7,12.5
3679,2021-11-04,758.0,759.9,12.9


In [46]:
from pandasql import sqldf
dfsql = lambda q: sqldf(q, globals())

query = '''SELECT
             strftime('%Y', Date) as Year
           ,AVG(xCO2_wet), AVG(xCO2_dry),AVG(temp)
           FROM
              ACID
           GROUP BY
              Year;
    '''
dfsql(query)

,Year,AVG(xCO2_wet),AVG(xCO2_dry),AVG(temp)
0,2011,412.700123,413.482646,14.622833
1,2012,354.722189,355.451015,13.562790
2,2013,412.318290,413.414727,13.519672
3,2014,397.626774,398.867695,14.208874
4,2015,409.133848,411.258230,13.992392
5,2016,362.048874,363.244202,13.505419
6,2017,602.303068,603.335425,6.948886
7,2018,400.815809,401.951247,15.791525
8,2019,465.976021,467.183259,13.116575
9,2020,455.964825,457.025447,12.257093


In [43]:
from pandasql import sqldf
dfsql = lambda q: sqldf(q, globals())

query = '''SELECT
             strftime('%Y', Date) as Year,
             strftime('%m', Date) as Month
           ,AVG(xCO2_wet), AVG(xCO2_dry),AVG(temp)
           FROM
              ACID
           GROUP BY
              Year, Month;
    '''
dfsql(query)

,Year,Month,AVG(xCO2_wet),AVG(xCO2_dry),AVG(temp)
0,2011,06,290.322892,290.900602,17.037349
1,2011,07,306.602419,307.485081,19.508065
2,2011,08,278.089919,279.088710,20.674194
3,2011,09,322.325417,323.362917,19.001250
4,2011,10,488.115041,488.947561,12.552016
...,...,...,...,...,...
96,2021,07,336.080645,337.400806,21.815726
97,2021,08,332.445161,333.681048,20.835887
98,2021,09,374.285000,375.335000,16.887917
99,2021,10,645.453061,646.765306,12.191429


In [44]:
from pandasql import sqldf
dfsql = lambda q: sqldf(q, globals())

query = '''SELECT
            strftime(Date) as Date,
           AVG(xCO2_wet), AVG(xCO2_dry),AVG(temp)
           FROM
              ACID
           GROUP BY
              Date;
    '''
dfsql(query)

,Date,AVG(xCO2_wet),AVG(xCO2_dry),AVG(temp)
0,2011-06-10 00:00:00.000000,261.366667,261.766667,15.4000
1,2011-06-11 00:00:00.000000,273.700000,274.100000,14.7375
2,2011-06-12 00:00:00.000000,271.275000,271.762500,17.6625
3,2011-06-13 00:00:00.000000,257.975000,258.500000,17.8375
4,2011-06-14 00:00:00.000000,274.475000,274.912500,15.8750
...,...,...,...,...
2873,2021-10-31 00:00:00.000000,815.840000,817.120000,9.7400
2874,2021-11-01 00:00:00.000000,803.050000,804.650000,11.7000
2875,2021-11-03 00:00:00.000000,777.800000,779.500000,12.0500
2876,2021-11-04 00:00:00.000000,761.850000,763.587500,12.4875
